In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc_version = 'gcc4'
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

Using TensorFlow backend.


(2156208, 2) (1724990, 2) (1773289, 2) (2224199, 2)
reset_index 완료


,bin,label
0,55,1
1,89,0
2,e5,0
3,53,0
4,83,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()),len(gcc6_1_32['bin'].unique()))
print(len(gcc6_2_32['bin'].unique()),len(gcc6_3_32['bin'].unique()))

256 256
256 256


In [3]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2156208, 257) (1724990, 257) (1773289, 257) (2224199, 257)


In [4]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['label'].value_counts(), '\n', gcc6_1_32_onehot['label'].value_counts())
print(gcc6_2_32_onehot['label'].value_counts(), '\n', gcc6_3_32_onehot['label'].value_counts())

0    2151901
1       4307
Name: label, dtype: int64 
 0    1721737
1       3253
Name: label, dtype: int64
0    1770091
1       3198
Name: label, dtype: int64 
 0    2221208
1       2991
Name: label, dtype: int64


In [5]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['bin'].value_counts()

00    322630
ff    186095
8b    155808
45    113507
89    100993
       ...  
a9       269
9e       266
96       252
b1       252
a6       239
Name: bin, Length: 256, dtype: int64

In [6]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
# left_idx, right_idx = -10, 11 # 10개씩
left_idx, right_idx = 0, 11 # 10개씩
# left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 47117


In [7]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 35160


In [8]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 35091


In [9]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 32828


In [10]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['label'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['label'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(47117, 256) (47117,)
(35160, 256) (35160,)
(35091, 256) (35091,)
(32828, 256) (32828,)


In [11]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(47117, 1, 256) (47117, 1, 1)
(35160, 1, 256) (35160, 1, 1)
(35091, 1, 256) (35091, 1, 1)
(32828, 1, 256) (32828, 1, 1)


In [12]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc6_0123_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_0123_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_0123_32.shape, y_gcc6_0123_32.shape)

(150196, 1, 256) (150196, 1, 1)


In [13]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(x_gcc6_0123_32, y_gcc6_0123_32):
    
    xInput = Input(batch_shape=(None, 1, 256)) # 
    xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
    xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #print(model1.summary())
    print('======Training stage======')
    model1.fit(x_gcc6_0123_32[train],
               y_gcc6_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(x_gcc6_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
135176/135176 [==============================] - 18s 135us/step - loss: 0.1635 - accuracy: 0.9478s - loss: 0
Epoch 2/3
135176/135176 [==============================] - 19s 142us/step - loss: 0.1386 - accuracy: 0.9526
Epoch 3/3
135176/135176 [==============================] - 20s 147us/step - loss: 0.1379 - accuracy: 0.9526


accuracy_score 0.9984021304926765
recall_score 1.0
precision_score 0.9827833572453372
f1_score 0.9913169319826338
======Training stage======
Epoch 1/3
135176/135176 [==============================] - 20s 151us/step - loss: 0.1624 - accuracy: 0.9477
Epoch 2/3
135176/135176 [==============================] - 19s 140us/step - loss: 0.1387 - accuracy: 0.9527
Epoch 3/3
135176/135176 [==============================] - 20s 148us/step - loss: 0.1378 - accuracy: 0.9527


accuracy_score 0.998202396804261
recall_score 1.0
precision_score 0.9807555238774056
f1_score 0.9902842749190356
======Training stage======
Epoch 1/3
135176/135176 [==============================] - 22s 166us/step - loss: 0.1639 - accuracy: 0.9473
Epoch 2/3
135176/135176 [==============================] - 22s 162us/step - loss: 0.1388 - accuracy: 0.9526
Epoch 3/3
135176/135176 [==============================] - 19s 138us/step - loss: 0.1376 - accuracy: 0.9526


accuracy_score 0.9984021304926765
recall_score 1.0
precision_score 0.9828203292770222
f1_score 0.9913357400722022
======Training stage======
Epoch 1/3
135176/135176 [==============================] - 21s 154us/step - loss: 0.1503 - accuracy: 0.9538
Epoch 2/3
135176/135176 [==============================] - 19s 142us/step - loss: 0.1255 - accuracy: 0.9584
Epoch 3/3
135176/135176 [==============================] - 15s 109us/step - loss: 0.1249 - accuracy: 0.9586


accuracy_score 0.9444074567243675
recall_score 0.40188269370021723
precision_score 0.9840425531914894
f1_score 0.5706940874035991
======Training stage======
Epoch 1/3
135176/135176 [==============================] - 27s 203us/step - loss: 0.1484 - accuracy: 0.9546
Epoch 2/3
135176/135176 [==============================] - 23s 168us/step - loss: 0.1234 - accuracy: 0.9599
Epoch 3/3
135176/135176 [==============================] - 20s 147us/step - loss: 0.1227 - accuracy: 0.9600


...

accuracy_score 0.9360852197070573
recall_score 0.3142650253439537
precision_score 0.970917225950783
f1_score 0.47483588621444206
======Training stage======
Epoch 1/3
135177/135177 [==============================] - 29s 218us/step - loss: 0.1484 - accuracy: 0.9542
Epoch 2/3
135177/135177 [==============================] - 21s 154us/step - loss: 0.1245 - accuracy: 0.9596
Epoch 3/3
135177/135177 [==============================] - 20s 146us/step - loss: 0.1236 - accuracy: 0.9596


accuracy_score 0.935348558492576
recall_score 0.30701754385964913
precision_score 0.9480812641083521
f1_score 0.46383213694091663
======Training stage======
Epoch 1/3
135177/135177 [==============================] - 18s 137us/step - loss: 0.1483 - accuracy: 0.9541
Epoch 2/3
135177/135177 [==============================] - 19s 138us/step - loss: 0.1250 - accuracy: 0.9589
Epoch 3/3
135177/135177 [==============================] - 25s 182us/step - loss: 0.1243 - accuracy: 0.9590s - loss: 0.1


accuracy_score 0.9377455223383714
recall_score 0.31921110299488675
precision_score 0.9931818181818182
f1_score 0.4831398562741846
======Training stage======
Epoch 1/3
135177/135177 [==============================] - 24s 178us/step - loss: 0.1494 - accuracy: 0.9541
Epoch 2/3
135177/135177 [==============================] - 19s 140us/step - loss: 0.1252 - accuracy: 0.9585
Epoch 3/3
135177/135177 [==============================] - 20s 144us/step - loss: 0.1245 - accuracy: 0.9587


accuracy_score 0.9440042612690592
recall_score 0.39400584795321636
precision_score 0.9782214156079855
f1_score 0.5617509119332985
======Training stage======
Epoch 1/3
135177/135177 [==============================] - 20s 148us/step - loss: 0.1495 - accuracy: 0.9536s -
Epoch 2/3
135177/135177 [==============================] - 22s 163us/step - loss: 0.1260 - accuracy: 0.9582
Epoch 3/3
135177/135177 [==============================] - 28s 204us/step - loss: 0.1252 - accuracy: 0.9583


accuracy_score 0.9462014781277049
recall_score 0.4198975859546452
precision_score 0.9745331069609507
f1_score 0.5869120654396728

K-fold cross validation Accuracy: [0.9984021304926765, 0.998202396804261, 0.9984021304926765, 0.9444074567243675, 0.9318908122503329, 0.9360852197070573, 0.935348558492576, 0.9377455223383714, 0.9440042612690592, 0.9462014781277049]

K-fold cross validation Recall: [1.0, 1.0, 1.0, 0.40188269370021723, 0.27298850574712646, 0.3142650253439537, 0.30701754385964913, 0.31921110299488675, 0.39400584795321636, 0.4198975859546452]

K-fold cross validation Precision: [0.9827833572453372, 0.9807555238774056, 0.9828203292770222, 0.9840425531914894, 0.9718670076726342, 0.970917225950783, 0.9480812641083521, 0.9931818181818182, 0.9782214156079855, 0.9745331069609507]

K-fold cross validation F1-Score: [0.9913169319826338, 0.9902842749190356, 0.9913357400722022, 0.5706940874035991, 0.42624789680314085, 0.47483588621444206, 0.46383213694091663, 0.4831398562741846, 0.561750

In [14]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9570689966699083
10-Fold Cross_validation. Recall : 0.5429268305553695
10-Fold Cross_validation. Precision : 0.9767203602073777
10-Fold Cross_validation. F1-Score : 0.6540349787983126
